In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config, sub_root_path
from bcnf.train import Trainer
from bcnf import CondRealNVP_v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'videos_CNN_LSTM_large'

In [4]:
config_path_pattern = os.path.join("{{BCNF_ROOT}}", "configs", "runs", f"{MODEL_NAME}.yaml")
config_path = sub_root_path(config_path_pattern)
print(f'Loading config from {config_path}')

config = load_config(config_path, verify=False)

Loading config from /home/psaegert/Projects/bcnf/configs/runs/trajectory_FC_small.yaml


In [5]:
model = CondRealNVP_v2.from_config(config).to(device)

print(f'{model.n_params:,}')

128,257


In [6]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 4/4 [00:28<00:00,  7.22s/it, file=fixed_data_render_2s_15FPS_4.pkl]


Using trajectories data for training. Shapes:
Conditions: [torch.Size([4000, 30, 3])]
Parameters: torch.Size([4000, 19])


In [7]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -29.4424 - Val: -28.2919 (avg: -32.4831, min: -32.5718) | lr: 2.00e-04 - Patience: 15/500 - z: (-0.1267 ± 0.5186) ± (0.9246 ± 0.2986):   6%|▌         | 2756/50000 [29:05<7:48:26,  1.68it/s] 

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path_pattern}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_FC_large
